Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = ""
COLLABORATORS = "Aitor Espinós & Guillem Perales"

---

# Python assignment

The python assignment you can do either on your own or with one other student (i.e. max group size is 2 students).

The first cell of your notebook, should contain a table with the names and SNRs and ANRs of the group members, like so

|Name|SNR|ANR|
|----|---|----|
|Aitor Espinós|2069780|u202365|
|Guillem Perales|2066999|u725893|



See [the webpage](https://janboone.github.io/applied-economics/#org065a005) for details of what we expect to see in this assignment.



# Research question (0.5 points)

Formulate the research question: what question do you want to answer in this assignment?

1. Do community-based societies lead to a better allocation of resources, so, to higher population?
2. Do societies tend to cluster into communities?
3. Does the individual capacity/ability influence the incentive to cluster into communities?
4. Is there a steady state? Is it unique for each endowment?

# Motivation (0.5 points)

Motivate why this question is interesting.

YOUR ANSWER HERE

# Method (0.5 points)

If you do a theory notebook (without external data; but perhaps with data that you simulate yourself), explain the methods that you use for the theory:

* is there an optimization problem underlying the notebook (like a firm maximizing profits)?
* do you calculate an equilibrium?
* what model do you use?

# [optional] Data [0.5 points]

**These 0.5 points are instead of the Method points above**

If you use data for your assignment, the Method part above does not apply. Instead,

* explain where your data come from (give a link or the code to download the data if you use an API like [wbdata](https://wbdata.readthedocs.io/en/stable/));
* explain the data cleaning and data normalization steps that you use;
* make sure that the data is in your github repository as well (or explain why this is not possible) so that we can replicate your analysis;
* what statistical methods do you use with the data.


1. The simulation starts with 'n' number of individuals. Each individual 'i' is part of a community 'c', and has a level of ability to find food 'a'.
2. Each period 'p', each individual finds 'f' pieces of food (where 'f' is a list (of random length depending on 'a') of different (but similar) pieces of food (which can go from 0 to 'y')).
3. If a specific piece of food is included inside the 'f' of 2 or more individuals:
    1. Individuals of different communities will fight each other resulting in a probability 'd' of dying for each individual fought.
    2. The piece of food will be randomly allocated to one of the individuals that survived.
4. If in a community there is an individual with more than 1 piece of food and an individual with less than 1 piece of food, the one with more gives 1 piece to the one with less.
5. Individuals with 0 pieces of food die.
6. Each individual has a probability 'l' of leaving its community.
7. Individuals that leave have a probability 'j' of joining another random community, or, otherwise, creating its own community.
7. Each individual has a probability 'r' of reproducing, creating a new individual with the same characteristics of its progenitor.

# Preview of the answers (0.5 points)

Summarize the results that you find and the answer to your research question.

YOUR ANSWER HERE

# Main assumptions (0.5 points)

What are the main assumptions that you need to answer the question? If you do a theory notebook, what are the main assumptions of your model? If you use data, what assumptions are needed by your estimation method.

1. No technological growth.
2. Closed frontiers.
3. Individuals have no information on other individuals.
4. Leaving or joining a community is exogenous.

# Python code (6.5 points)


Give the python code in code cells and use markdown cells to explain why you code things in this way and what the outcomes are of the code cells.

Note that the explanation of your code is at least as important as the python code itself.

Create as many python and markdown cells as you need to explain things well.

First, we import all the libraries we will need for the simulation and its further analysis:

In [2]:
import random as rd
import numpy as np
import pandas as pd
import matplotlib.pyplot as mp
import sqlite3 as sq

Then, we will start by setting the number of periods that the simulation will have:

In [3]:
p=100

We continue by setting the initial number of individuals that will exist in the simulation:

In [4]:
n=10

We set the initial number of different communities:

In [5]:
c=3

The minimum and the maximum level of ability to find food that an individual can have in the simulation:

In [6]:
min_a=1
max_a=3

And the minimum and maximum amount of food with which each individual can start with:

In [7]:
min_s=0
max_s=1

Now we will assign a value, for period 1, for each characteristic that every individual has.

We will create a 2-dimensional array for every characteristic, in which the x-axis will represent the 'period' and the y-axis the 'individual':

In [8]:
aliveness=[[]]
community=[[]]
ability=[[]]
savings=[[]]

The characteristic 'aliveness' is a dummy variable that takes the value of 0 if the individual starts the period dead, or the value of 1 if the individual starts the period alive.

For period 1, it will take the value of 1 for each individual as all of them will start the simulation alive:

In [9]:
for i in range(n):
    aliveness[0].append(1)
print(aliveness)

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


The characteristic 'community' will take a random value from 1 to 'c' in period 1:

In [10]:
for i in range(n):
    community[0].append(rd.randint(1,c))
print(community)

[[2, 3, 1, 3, 2, 3, 3, 2, 3, 3]]


The characteristic 'ability' will take a random value from 'min_a' to 'max_a' in period 1:

In [11]:
for i in range(n):
    ability[0].append(rd.randint(min_a,max_a))
print(ability)

[[3, 3, 2, 1, 1, 1, 2, 1, 2, 3]]


The characteristic 'savings' will take random value from 'min_s to 'max_s' in period 1:

In [12]:
for i in range(n):
    savings[0].append(rd.randint(min_s,max_s))
print(savings)

[[1, 1, 0, 0, 1, 1, 0, 0, 1, 1]]


The next step is to set the maximum amount of food that can be found by the total of individuals each period:

In [13]:
max_f=20

Then, we will randomly (but depending on 'ability') assign how many pieces of food does each individual find in the 1st period.

For this, we create another 2-dimensional array, this time for the variable 'found_food'. We then assign a random number from 0 to 'ability' for each alive individual in period 1:

In [14]:
found_food=[[]]
for i in range(len(aliveness[0])):
    if aliveness[0][i]==1:
        found_food[0].append(rd.randint(0,ability[0][i]))
    else:
        found_food[0].append(0)
print(found_food)

[[0, 0, 2, 1, 0, 0, 1, 1, 2, 2]]


If the sum of food found by all individuals in the period happened to exceed the maximum amount of food 'max_f', the difference between the sum and 'max_f' would be randomly taken away, unit by unit, from all individuals whose 'found_food' in period 1 is higher than 0:

In [15]:
while max_f<sum(found_food[0]):
    chosen=rd.randint(0,len(found_food[0])-1)
    if 0<found_food[0][chosen]:
        found_food[0][chosen]-=1
print(found_food)

[[0, 0, 2, 1, 0, 0, 1, 1, 2, 2]]


The next step is to sum the savings that each individual started the period with, plus the amount of food found by each individual during the period. We store the resulting values as the savings for the next period (the second period in this case):

In [16]:
savings.append([])
for i in range(len(aliveness[0])):
    savings[1].append(savings[0][i]+found_food[0][i])
print(savings)

[[1, 1, 0, 0, 1, 1, 0, 0, 1, 1], [1, 1, 2, 1, 1, 1, 1, 1, 3, 3]]


Then, if an individual would happen to die because it has nothing to eat, someone from the same community would share a piece of food with him/her (younger individuals would be prioritized in case there is not enough for everyone):

In [17]:
for i in reversed(range(len(aliveness[0]))):
    if savings[1][i]<1:
        for j in range(len(aliveness[0])):
            if community[0][j]==community[0][i] and 1<savings[1][j]:
                savings[1][j]-=1
                savings[1][i]+=1
                break
print(savings)

[[1, 1, 0, 0, 1, 1, 0, 0, 1, 1], [1, 1, 2, 1, 1, 1, 1, 1, 3, 3]]


And at the end of the period, individuals that have food would consume 1 piece to survive, and individuals that don't have food would die:

In [18]:
aliveness.append([])
for i in range(len(aliveness[0])):
    if 0<savings[1][i]:
        savings[1][i]-=1
        aliveness[1].append(1)
    else:
        aliveness[1].append(0)
print(savings)
print(aliveness)

[[1, 1, 0, 0, 1, 1, 0, 0, 1, 1], [0, 0, 1, 0, 0, 0, 0, 0, 2, 2]]
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


Then, there is a chance 'l' that individuals leave their community, and if they do so, they will join any other community or create another one, all with equal probability:

In [19]:
l=0.1
community.append([])
for i in range(len(aliveness[0])):
    community[1].append(community[0][i])
    if rd.uniform(0,1)<l:
        while community[1][i]==community[0][i]:
            community[1][i]=rd.randint(1,max(community[0])+1)
print(community)

[[2, 3, 1, 3, 2, 3, 3, 2, 3, 3], [4, 3, 2, 3, 2, 1, 3, 2, 3, 3]]


Finally, at the end of the period, each alive individual will have a chance 'r' of reproducing. The newborn individual will share the same characteritics as his/her progenitor, but with 'savings' equal to 0:

In [ ]:
r=0.1
ability.append([])
for i in range(len(aliveness[0])):
    if rd.uniform(0,1)<1:
        

# Sensitivity analysis (0.5 points)

If you do a theory notebook, program different values for the parameters, use different functional forms and solve your model again for these new values. Show that the outcomes are robust to such changes.

If you are using data, how sensitive are your results to outliers in the data, different choices when cleaning the data, different functional forms for the relations that you estimate.


In [20]:
# YOUR CODE HERE

# Discussion and conclusion (0.5 points)

What did you find in the analysis above; what is the answer to the question you started out with.

What are weaknesses of your approach that can be improved upon in future research (e.g. in your thesis).


We are supposing a closed economy with no technology improvement.